# Simple Moving Average Cross Over Strategy - 3 SMA's

- Compute three moving averages of 20, 40, and 80.
- Go long when the price crosses above all three moving averages.
- Exit the long position when the price crosses below any of the three moving averages.
- Go short when the price crosses below all three moving averages.
- Exit the short position when the price crosses above any of the three moving averages.
- Optional: Optimize all three moving averages

Import the required libraries

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

In [ ]:
# Create class
class backtest_cross_over:
    
    # Create object attributes and methods
    def __init__(self,ticker,start,end,sma,mma,lma):
        self.ticker = ticker
        self.start = start
        self.end = end
        self.sma = sma
        self.mma = mma
        self.lma = lma
        
        self.fetch_data()
        self.indicators()
        self.signals()
        self.positions()
        self.returns()
    
    #To fetch the data from yahoo finance
    def fetch_data(self):
        self.df = yf.download(self.ticker,self.start,self.end)
     
    # To compute the indicators necessary for the Strategy
    def indicators(self):
        self.df['sma'] = self.df['Adj Close'].rolling(window=self.sma,center=False).mean()
        self.df['mma'] = self.df['Adj Close'].rolling(window=self.mma,center=False).mean()
        self.df['lma'] = self.df['Adj Close'].rolling(window=self.lma,center=False).mean()
        
    # To generate the long and short trading signals    
    def signals(self):
        self.df['signal'] = np.where((self.df['Adj Close'] > self.df['sma']) & (self.df['Adj Close'] > self.df['mma']) & (self.df['Adj Close'] > self.df['lma']),1,0)
        self.df['signal'] = np.where((self.df['Adj Close'] < self.df['sma']) & (self.df['Adj Close'] > self.df['lma']),0,self.df['signal'])

        self.df['signal'] = np.where((self.df['Adj Close'] < self.df['sma']) & (self.df['Adj Close'] < self.df['mma']) & (self.df['Adj Close'] < self.df['lma']),-1,self.df['signal'])
        self.df['signal'] = np.where((self.df['Adj Close'] > self.df['sma']) & (self.df['Adj Close'] < self.df['lma']),0,self.df['signal'])
    
    #To generate the positions
    def positions(self):
        self.df['positions'] = self.df['signal'].shift(1)
    
    # To generate the Buy & Hold and Strategy returns
    def returns(self):
        self.df['BnH returns'] = self.df['Adj Close'].pct_change()
        self.df['Strategy Returns'] = self.df['BnH returns']*self.df['positions']
        BnH = (self.df['BnH returns']+1).cumprod()[-1]
        Strat_returns = (self.df['Strategy Returns']+1).cumprod()[-1]
        print('Total Strategy Returns:',(self.df['Strategy Returns']+1).cumprod()[-1])
        return Strat_returns

In [ ]:
#Fix the backtesting period of 3 years
end = pd.datetime.now().date()
start = end-pd.Timedelta(days=3*252)

In [ ]:
#To generate the returns and validate by optimising the moving average parameters across different indices

sma_list = range(10,30,5)
mma_list = range(30,75,10)
lma_list = range(80,160,10)
indice_list=['^NSEI','SPY','HSI','^CNXIT','^NSEBANK','^HSI','^GSPC']

index_name =[]
sma=[]
mma=[]
lma=[]

net_returns=[]

for i in lma_list:
    for j in mma_list:
        for k in sma_list:
            for l in indice_list:
                print('For',i,j,k,l)
                a = backtest_cross_over(l, start1, end1, i, j, k)
                lma.append(i)
                mma.append(j)
                sma.append(k)
                index_name.append(l)
                net_returns.append(a.returns())

In [ ]:
#Convert the results into the dataframe
results = pd.DataFrame({'long_ma':lma,'med_ma': mma,'short_ma':sma,'index' :index_name,'net_returns':net_returns})


In [ ]:
#To identify the optimum parameters generating greater returns
results.sort_values(by='net_returns',ascending=False,inplace=True)

In [ ]:
results.head(20)

As per the above table, CNXIT generates good and consistent returns with the moving average of 10,70,120 

# Exponential Moving Average Cross Over

Try Exponential moving average instead of Simple moving average to validate the strategy. We can reuse the same parent class and create child class only to replace for EMA calculation

In [ ]:
class backtest_EMA_cross_over(backtest_cross_over):
    
    def indicators(self):
        self.df['sma'] = self.df['Adj Close'].ewm(span=self.sma,adjust=False).mean()
        self.df['mma'] = self.df['Adj Close'].ewm(span=self.mma,adjust=False).mean()
        self.df['lma'] = self.df['Adj Close'].ewm(span=self.lma,adjust=False).mean()
        

Lets try CNXIT with exponential moving average of same parameters to check the returns are higher

In [ ]:
IT_Nifty_EMA_Cross_over = backtest_EMA_cross_over('^CNXIT',start1,end1,10,70,120)

In [ ]:
IT_Nifty_SMA_Cross_over = backtest_cross_over('^CNXIT',start1,end1,10,70,120)